<a href="https://colab.research.google.com/github/jdilger/TensorFlowNotebooks/blob/master/AgroForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# note delete for share
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [ ]:
# Use logging to maintain more detailed information for reproducibility 
import logging

def tfLog():
  logging.basicConfig(level=logging.DEBUG, filename='myapp.log',
                      format='%(asctime)s %(levelname)s:%(message)s')
  try:
    logging.debug('######################################')
    logging.debug('Config Settings')
    logging.debug('######################################')
    logging.debug("Bucket:%s",BUCKET) 
    logging.debug("Folder:%s",FOLDER)
    logging.debug('Training base:%s',TRAINING_BASE)
    logging.debug('Eaval base:%s',EVAL_BASE) 
    logging.debug('Band order:%s',BANDS)
    logging.debug('Response:%s',RESPONSE)
    logging.debug('Features:%s',FEATURES)
    logging.debug('Kernal size:%d',KERNEL_SIZE)
    logging.debug('FEATURES_DICT:%s',FEATURES_DICT)
    logging.debug('Training size:%d',TRAIN_SIZE)
    logging.debug('Eval size:%d',EVAL_SIZE)
    logging.debug('batch size:%d',BATCH_SIZE)
    logging.debug('Epochs:%d',EPOCHS)
    logging.debug('Buffer size:%d',BUFFER_SIZE) 
    logging.debug('Optimizer:%s',OPTIMIZER) 
    # logging.debug('Loss:',LOSS)
    # logging.debug('Other metrics:',METRICS)
  except Exception as e:
    print('logging failed')
    print(e.args)

In [ ]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Import, authenticate and initialize the Earth Engine library.
import ee
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [ ]:
!pip install tensorflow==2.2.0

In [ ]:
# Tensorflow setup.
# %tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)
# tf.enable_eager_execution()


In [ ]:
# INSERT YOUR BUCKET HERE:
BUCKET = 'tf-agro-forest'

In [ ]:
from tensorflow.python.keras import backend
# dice coeff and dice loss from Biplov
def dice_coeff(y_true, y_pred, smooth=1):
    y_true_f = backend.flatten(y_true)
    y_pred_f = backend.flatten(y_pred)
    intersection = backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (backend.sum(y_true_f) + backend.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

# soft dice loss function from Kel
# based on https://arxiv.org/pdf/1707.03237.pdf 
def dice_loss_soft(y_true, y_pred, smooth=1):
    intersection = backend.sum(backend.abs(y_true * y_pred), axis=-1)
    true_sum = backend.sum(backend.square(y_true),-1) 
    pred_sum = backend.sum(backend.square(y_pred),-1)
    return 1 - ((2. * intersection + smooth) / (true_sum + pred_sum + smooth))

## Set other global variables

In [ ]:
from tensorflow.python.keras import metrics
# Specify names locations for outputs in Cloud Storage. 
FOLDER = 'jdilger/training_data' 
TRAINING_BASE = 'training_patches_updated_noconst_20_kernalsize_4'
EVAL_BASE = 'testing_patches_updated_noconst_20_kernalsize_4'

# Specify inputs (Landsat bands) to the model and the response variable.
  
BANDS =['blue_stdDev', 'green_stdDev', 'red_stdDev', 're1_stdDev', 're2_stdDev', 're3_stdDev',
        'nir_stdDev', 'swir1_stdDev', 'swir2_stdDev', 'blue_stdDev_1', 'green_stdDev_1', 'red_stdDev_1',
        're1_stdDev_1', 're2_stdDev_1', 're3_stdDev_1', 'nir_stdDev_1', 'swir1_stdDev_1', 'swir2_stdDev_1',
        'blue_stdDev_2', 'green_stdDev_2', 'red_stdDev_2', 're1_stdDev_2', 're2_stdDev_2', 're3_stdDev_2', 
        'nir_stdDev_2', 'swir1_stdDev_2', 'swir2_stdDev_2', 'blue_stdDev_3', 'green_stdDev_3', 'red_stdDev_3',
        're1_stdDev_3', 're2_stdDev_3', 're3_stdDev_3', 'nir_stdDev_3', 'swir1_stdDev_3', 'swir2_stdDev_3', 
         'blue', 'green', 'red', 're1', 're2', 're3', 'nir', 'swir1', 'swir2', 'blue_1', 'green_1',
        'red_1', 're1_1', 're2_1', 're3_1', 'nir_1', 'swir1_1', 'swir2_1', 'blue_2', 'green_2', 'red_2', 're1_2',
        're2_2', 're3_2', 'nir_2', 'swir1_2', 'swir2_2', 'blue_3', 'green_3', 'red_3', 're1_3', 're2_3', 're3_3',
        'nir_3', 'swir1_3', 'swir2_3', 'blue_4', 'green_4', 'red_4', 're1_4', 're2_4', 're3_4',
        'nir_4', 'swir1_4', 'swir2_4', 'blue_1_1', 'green_1_1', 'red_1_1', 're1_1_1', 're2_1_1',
        're3_1_1', 'nir_1_1', 'swir1_1_1', 'swir2_1_1', 'blue_2_1', 'green_2_1', 'red_2_1', 're1_2_1',
        're2_2_1', 're3_2_1', 'nir_2_1', 'swir1_2_1', 'swir2_2_1', 'blue_3_1', 'green_3_1', 'red_3_1', 're1_3_1',
        're2_3_1', 're3_3_1', 'nir_3_1', 'swir1_3_1', 'swir2_3_1', 'VH', 'VV', 'VH_1', 'nd', 'VH_2', 'VV_1',
        'VV_2', 'VH_3', 'VV_3', 'VH_1_1', 'nd_1', 'VH_2_1', 'VV_1_1', 'VV_2_1', 'VH_stdDev', 'VV_stdDev',
        'VH_1_stdDev', 'nd_stdDev', 'VH_2_stdDev', 'VV_1_stdDev', 'VV_2_stdDev']

RESPONSE = ['cocao','forest','seasonalAg'
          ,'urban','water','banana','savana','orchard','mine','shrubland','sparseTree','bare'
          ,'grassland','secondaryForest','nodata']
FEATURES = BANDS + RESPONSE

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 4
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# Sizes of the training and evaluation datasets.
TRAIN_SIZE = 7000
EVAL_SIZE = 500

# Specify model training parameters.
BATCH_SIZE = 30
EPOCHS = 10
BUFFER_SIZE = 7000
OPTIMIZER = 'Adam'
LOSS = dice_loss
# update this
METRICS =   [metrics.get('RootMeanSquaredError'),
    metrics.get('MeanAbsoluteError'),
    metrics.get('Accuracy'),
    dice_coeff,]

In [ ]:
tfLog()

In [ ]:

!cat myapp.log

# Training data

Load the data exported from Earth Engine into a `tf.data.Dataset`.  The following are helper functions for that.

In [ ]:
def parse_tfrecord(example_proto):
  """The parsing function.
  Read a serialized example into the structure defined by FEATURES_DICT.
  Args:
    example_proto: a serialized Example.
  Returns: 
    A dictionary of tensors, keyed by feature name.
  """
  return tf.io.parse_single_example(example_proto, FEATURES_DICT)


def to_tuple(inputs):
  """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
  Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
  Args:
    inputs: A dictionary of tensors, keyed by feature name.
  Returns: 
    A dtuple of (inputs, outputs).
  """
  inputsList = [inputs.get(key) for key in FEATURES]
  stacked = tf.stack(inputsList, axis=0)
  # Convert from CHW to HWC
  stacked = tf.transpose(stacked, [1, 2, 0])
  return stacked[:,:,:len(BANDS)], stacked[:,:,len(BANDS):]


def get_dataset(pattern,flip=False):
  """Function to read, parse and format to tuple a set of input tfrecord files.
  Get all the files matching the pattern, parse and convert to tuple.
  Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
  Returns: 
    A tf.data.Dataset
  """
  glob = tf.io.gfile.glob(pattern)
  dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
  dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
  dataset = dataset.map(to_tuple, num_parallel_calls=5)
  if flip:
    dataset = dataset.map(transform)
  return dataset

In [ ]:
# custom function to randomly augment the data during training
# from kels notebooks
# adapted with python random rather than tf. Not sure if it works..
import random
def transform(features,labels):
    x = random.random()
    # flip image on horizontal axis
    if round(x,2) < 0.12: 
        feat = tf.image.flip_left_right(features)
        labl = tf.image.flip_left_right(labels)
    # flip image on vertical axis
    elif round(x,2) >=0.12 and round(x,2) < 0.24:
        feat = tf.image.flip_up_down(features)
        labl = tf.image.flip_up_down(labels)
    # transpose image on bottom left corner
    elif round(x,2) >=0.24 and round(x,2) < 0.36:
        feat = tf.image.flip_left_right(tf.image.flip_up_down(features))
        labl = tf.image.flip_left_right(tf.image.flip_up_down(labels))
    # rotate to the left 90 degrees
    elif round(x,2) >=0.36 and round(x,2) < 0.48:
        feat = tf.image.rot90(features,k=1)
        labl = tf.image.rot90(labels,k=1)
    # rotate to the left 180 degrees
    elif round(x,2) >=0.48 and round(x,2) < 0.60:
        feat = tf.image.rot90(features,k=2)
        labl = tf.image.rot90(labels,k=2)
    # rotate to the left 270 degrees
    elif round(x,2) >=0.60 and round(x,2) < 0.72:
        feat = tf.image.rot90(features,k=3)
        labl = tf.image.rot90(labels,k=3)
    # transpose image on bottom right corner
    elif round(x,2) >=0.72 and round(x,2) < 0.84:
        feat = tf.image.flip_left_right(tf.image.rot90(features,k=2))
        labl = tf.image.flip_left_right(tf.image.rot90(labels,k=2))
    else:
        feat = features
        labl = labels

    return feat,labl

Use the helpers to read in the training dataset.  Print the first record to check.

In [ ]:
def get_training_dataset():
	"""Get the preprocessed training dataset
  Returns: 
    A tf.data.Dataset of training data.
  """
	glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + TRAINING_BASE + '*'
	dataset = get_dataset(glob,flip=False)
	dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
	return dataset

training = get_training_dataset()

print(iter(training.take(1)).next())

# Evaluation data

Now do the same thing to get an evaluation dataset.  Note that unlike the training dataset, the evaluation dataset has a batch size of 1, is not repeated and is not shuffled.

In [ ]:
def get_eval_dataset():
	"""Get the preprocessed evaluation dataset
  Returns: 
    A tf.data.Dataset of evaluation data.
  """
	glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + EVAL_BASE + '*'
	dataset = get_dataset(glob)
	dataset = dataset.batch(1).repeat()
	return dataset

evaluation = get_eval_dataset()

# Model

Here we use the Keras implementation of the U-Net model as found [in the TensorFlow examples](https://github.com/tensorflow/models/blob/master/samples/outreach/blogs/segmentation_blogpost/image_segmentation.ipynb).  The U-Net model takes 256x256 pixel patches as input and outputs per-pixel class probability, label or a continuous output.  We can implement the model essentially unmodified, but will use mean squared error loss on the sigmoidal output since we are treating this as a regression problem, rather than a classification problem.  Since impervious surface fraction is constrained to [0,1], with many values close to zero or one, a saturating activation function is suitable here.

In [ ]:
from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import models
from tensorflow.python.keras import metrics
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model

visible =  layers.Input(shape=[None, None, len(BANDS)])
# from stackexchange
enocded_imag = layers.Conv2D(64, (7, 7), activation='relu', padding='same')(visible)
enocded_imag = layers.MaxPooling2D((2, 2), padding='same')(enocded_imag)
enocded_imag = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(visible)
enocded_imag = layers.MaxPooling2D((2, 2), padding='same')(enocded_imag)
enocded_imag = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(enocded_imag)
enocded_imag = layers.MaxPooling2D((2, 2), padding='same')(enocded_imag)
enocded_imag = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(enocded_imag)
enocded_imag = layers.MaxPooling2D((2, 2), padding='same')(enocded_imag)

decoded_imag = layers.Conv2D(8, (2, 2), activation='relu', padding='same')(enocded_imag)
decoded_imag = layers.UpSampling2D((2, 2),interpolation='bilinear')(decoded_imag)
decoded_imag = layers.BatchNormalization()(decoded_imag)
decoded_imag = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(decoded_imag)
decoded_imag = layers.UpSampling2D((2, 2),interpolation='bilinear')(decoded_imag)
decoded_imag = layers.BatchNormalization()(decoded_imag)
decoded_imag = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(decoded_imag)
decoded_imag = layers.UpSampling2D((2, 2),interpolation='bilinear')(decoded_imag)
decoded_imag = layers.BatchNormalization()(decoded_imag)
decoded_imag = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(decoded_imag)


outBranch = layers.Conv2D(128, 3, activation='relu', 
                          padding='same',name="out_block_conv1")(decoded_imag)
outBranch = layers.SpatialDropout2D(rate=0.2,seed=1,name="out_block_spatialdrop")(outBranch)
outBranch = layers.BatchNormalization(name="out_block_batchnorm1")(outBranch)
outBranch = layers.Conv2D(len(RESPONSE), (1, 1), activation='relu')(outBranch)
outputs = layers.Activation("softmax")(outBranch)
model = models.Model(inputs=visible, outputs=outputs)
# summarize layers
print(model.summary())
# plot graph
plot_model(model, to_file='convolutional_neural_network.png')

In [ ]:
# small conv for 4x4
from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import models
from tensorflow.python.keras import metrics
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model

visible =  layers.Input(shape=[None, None, len(BANDS)])
# from stackexchange
enocded_imag = layers.Conv2D(128, (1, 1), activation='relu', padding='same')(visible)


outBranch = layers.BatchNormalization(name="out_block_batchnorm1")(enocded_imag)
outBranch = layers.Conv2D(len(RESPONSE), (1, 1), activation='relu')(outBranch)
outputs = layers.Activation("softmax")(outBranch)
model = models.Model(inputs=visible, outputs=outputs)
# summarize layers
print(model.summary())
# plot graph
plot_model(model, to_file='convolutional_neural_network.png')

In [ ]:

model.compile(
		optimizer=optimizers.Adam(lr=0.001), 
		loss=dice_loss_soft,
		metrics=[metrics.get('CategoricalAccuracy')])

In [ ]:
m = model
MODEL_FOLDER = 'tinyswig'
modelDir = 'gs://{}/{}/{}'.format(BUCKET,FOLDER,MODEL_FOLDER)

history = m.fit(
    x=training, 
    epochs=15,#EPOCHS, 
    steps_per_epoch=int(TRAIN_SIZE / 1), 
    validation_data=evaluation,
    validation_steps=EVAL_SIZE)


# m.save(modelDir, save_format='tf')


In [ ]:
m.save(modelDir, save_format='tf')

In [ ]:
# plot the results of model training
# get numpy and matplotlib.pyplot
# from kels notebooks
%pylab inline
fig, ax = plt.subplots(nrows=2, sharex=True, figsize=(10,5.5))

ax[0].plot(history.history['loss'],color='#1f77b4',label='Training Loss')
ax[0].plot(history.history['val_loss'],linestyle=':',marker='o',markersize=3,color='#1f77b4',label='Validation Loss')
ax[0].set_ylabel('Loss')
ax[0].set_ylim(0.0,0.4)
ax[0].legend()

ax[1].plot(history.history['categorical_accuracy'],color='#ff7f0e',label='Training Acc.')
ax[1].plot(history.history['val_categorical_accuracy'],linestyle=':',marker='o',markersize=3,color='#ff7f0e',label='Validation Acc.')
ax[1].set_ylabel('Accuracy')
ax[1].set_xlabel('Epoch')
ax[1].legend(loc="lower right")

ax[1].set_xticks(history.epoch)
ax[1].set_xticklabels(range(1,len(history.epoch)+1))
ax[1].set_xlabel('Epoch')
ax[1].set_ylim(0.0,1)

plt.legend()

# plt.savefig("/content/drive/My Drive/landsat_qa_samples/training.png",dpi=300,)

plt.show()

# Training the model

You train a Keras model by calling `.fit()` on it.  Here we're going to train for 10 epochs, which is suitable for demonstration purposes.  For production use, you probably want to optimize this parameter, for example through [hyperparamter tuning](https://cloud.google.com/ml-engine/docs/tensorflow/using-hyperparameter-tuning).

Note that the notebook VM is sometimes not heavy-duty enough to get through a whole training job, especially if you have a large buffer size or a large number of epochs.  You can still use this notebook for training, but may need to set up an alternative VM ([learn more](https://research.google.com/colaboratory/local-runtimes.html)) for production use.  Alternatively, you can package your code for running large training jobs on Google's AI Platform [as described here](https://cloud.google.com/ml-engine/docs/tensorflow/trainer-considerations).  The following code loads a pre-trained model, which you can use for predictions right away.

In [ ]:
from tensorflow.python.tools import saved_model_utils


meta_graph_def = saved_model_utils.get_meta_graph_def(modelDir, 'serve')
inputs = meta_graph_def.signature_def['serving_default'].inputs
outputs = meta_graph_def.signature_def['serving_default'].outputs

# Just get the first thing(s) from the serving signature def.  i.e. this
# model only has a single input and a single output.
input_name = None
for k,v in inputs.items():
  input_name = v.name
  break

output_name = None
for k,v in outputs.items():
  output_name = v.name
  break

# Make a dictionary that maps Earth Engine outputs and inputs to 
# AI Platform inputs and outputs, respectively.
import json
input_dict = "'" + json.dumps({input_name: "array"}) + "'"
output_dict = "'" + json.dumps({output_name: "class"}) + "'"

# Put the EEified model next to the trained model directory.
# TODO: add eeidied dir, project into to log, add output name
EEIFIED_DIR = '{}/eeified'.format(modelDir)
PROJECT = 'john-ee-282116'
print(input_dict,output_dict)
# You need to set the project before using the model prepare command.
!earthengine set_project {PROJECT}
!earthengine model prepare --source_dir {modelDir} --dest_dir {EEIFIED_DIR} --input {input_dict} --output {output_dict}

In [ ]:
modelDir

In [ ]:
import time 
MODEL_NAME = 'tinyswig'

VERSION_NAME = 'v' + str(int(time.time()))
print('Creating version: ' + VERSION_NAME)

!gcloud ai-platform models create {MODEL_NAME} --project {PROJECT}
!gcloud ai-platform versions create {VERSION_NAME} \
  --project {PROJECT} \
  --model {MODEL_NAME} \
  --origin {EEIFIED_DIR} \
  --runtime-version=2.2 \
  --framework "TENSORFLOW" \
  --python-version=3.7

In [ ]:
# Load a trained model. 
MODEL_DIR = 'gs://ee-tf/tahoe-ogfw-02292020/model-ogwf-256'
m = tf.contrib.saved_model.load_keras_model(MODEL_DIR)
help(m.summary())
